# Chapter 07-02: Transformer 기초

## 학습 목표
- Positional Encoding의 수식과 역할을 이해하고 직접 구현한다
- Transformer Encoder Block을 구성하는 요소를 구현한다
- 여러 Encoder Block을 쌓아 완전한 Transformer Encoder를 만든다
- IMDB 감성 분류에 Transformer를 적용하여 성능을 측정한다

## 목차
1. 핵심 수식
2. Positional Encoding 구현 및 시각화
3. Encoder Block 구현
4. Transformer Encoder 스택
5. IMDB 텍스트 분류 실험
6. 정리

---
### 🐣 초등학생을 위한 Transformer 친절 설명!

#### 🤔 RNN 대신 왜 Transformer?

| 문제점 | RNN/LSTM | Transformer |
|--------|----------|-------------|
| 긴 의존성 | 앞부분 기억 못함 | Attention으로 직접 연결! |
| 병렬 처리 | 순서대로만 처리 | 모든 위치 동시 처리! |
| 학습 속도 | 느림 | 훨씬 빠름 |

#### 🏗️ Transformer 핵심 구조

```
입력 문장
    ↓ Embedding (단어 → 벡터)
    ↓ Positional Encoding (위치 정보 추가)
    ↓ [Encoder 블록 × N]
       - Multi-Head Self-Attention
       - Add & Norm (잔차 연결)
       - Feed Forward
       - Add & Norm
    ↓ 최종 표현 (각 단어의 문맥 정보 포함!)
```

#### 📍 Positional Encoding이 왜 필요해요?

Attention은 위치 정보가 없어요! '나는 학교에 갔다'와
'갔다 학교에 나는'이 Attention에게는 **같아 보여요**!

> 💡 Positional Encoding = 각 위치에 **고유한 주파수 신호**를 더해줘요.
> 마치 우편번호처럼 각 단어에 위치 정보를 추가!

#### 🎭 Add & Norm (잔차 연결 + 정규화)

```
출력 = LayerNorm(x + Sublayer(x))
```
- **Residual(잔차)**: 입력을 직접 더해 기울기 소실 방지
- **LayerNorm**: 값 분포를 안정화해 학습 안정성 향상


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f'TensorFlow 버전: {tf.__version__}')

## 1. 핵심 수식

### Positional Encoding

Transformer는 순서 정보가 없으므로, 각 토큰의 위치(position)를 sin/cos 함수로 인코딩하여 임베딩에 더한다.

$$PE_{(pos,\,2i)} = \sin\!\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

$$PE_{(pos,\,2i+1)} = \cos\!\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

- $pos$: 시퀀스 내 토큰의 위치 인덱스 (0, 1, 2, ...)
- $i$: 차원 인덱스 (0, 1, ..., $d_{model}/2 - 1$)
- 짝수 차원에는 $\sin$, 홀수 차원에는 $\cos$ 적용
- 주기가 차원마다 다르므로 각 위치는 고유한 패턴을 가짐

### Feed-Forward Network (FFN)

각 Encoder Block 내부의 위치별 완전연결층:

$$\text{FFN}(x) = \max(0,\; xW_1 + b_1)W_2 + b_2$$

- 첫 번째 Linear → ReLU → 두 번째 Linear 구조
- 각 위치에 독립적으로 적용 (position-wise)
- 내부 차원($d_{ff}$)은 보통 $d_{model}$의 4배

## 2. Positional Encoding 구현 및 시각화

In [ ]:
def get_positional_encoding(max_seq_len, d_model):
    """
    Positional Encoding 행렬 계산
    
    Args:
        max_seq_len: 최대 시퀀스 길이
        d_model: 모델 임베딩 차원
    Returns:
        pe: shape = (1, max_seq_len, d_model)
    """
    # 위치 인덱스 벡터 (max_seq_len, 1)
    positions = np.arange(max_seq_len)[:, np.newaxis]  # (max_seq_len, 1)
    
    # 차원 인덱스 벡터 (1, d_model/2)
    dims = np.arange(0, d_model, 2)[np.newaxis, :]     # (1, d_model/2)
    
    # 분모 계산: 10000^(2i/d_model)
    div_term = np.power(10000.0, dims / d_model)       # (1, d_model/2)
    
    # Positional Encoding 행렬 초기화
    pe = np.zeros((max_seq_len, d_model))
    
    # 짝수 차원: sin
    pe[:, 0::2] = np.sin(positions / div_term)
    # 홀수 차원: cos
    pe[:, 1::2] = np.cos(positions / div_term)
    
    # 배치 차원 추가 → (1, max_seq_len, d_model)
    return pe[np.newaxis, :, :]


# 시각화
max_seq_len = 50
d_model = 64
pe = get_positional_encoding(max_seq_len, d_model)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 전체 PE 행렬 히트맵
ax1 = axes[0]
im = ax1.imshow(pe[0], aspect='auto', cmap='RdBu', vmin=-1, vmax=1)
ax1.set_xlabel('차원 인덱스 (d_model)', fontsize=11)
ax1.set_ylabel('위치 (position)', fontsize=11)
ax1.set_title(f'Positional Encoding 행렬\n(max_seq_len={max_seq_len}, d_model={d_model})', fontsize=11)
plt.colorbar(im, ax=ax1)

# 특정 차원에서의 sin/cos 파형
ax2 = axes[1]
positions_range = np.arange(max_seq_len)
for dim_idx in [0, 2, 4, 8, 16]:
    ax2.plot(positions_range, pe[0, :, dim_idx],
             label=f'dim={dim_idx} ({"sin" if dim_idx % 2 == 0 else "cos"})')
ax2.set_xlabel('위치 (position)', fontsize=11)
ax2.set_ylabel('PE 값', fontsize=11)
ax2.set_title('차원별 Positional Encoding 파형', fontsize=11)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

plt.suptitle('Positional Encoding 시각화', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print(f'PE 행렬 shape: {pe.shape}')  # (1, 50, 64)
print(f'위치 0의 첫 8차원: {pe[0, 0, :8].round(4)}')
print(f'위치 1의 첫 8차원: {pe[0, 1, :8].round(4)}')

## 3. Encoder Block 구현

Encoder Block 구조:
```
입력
 ├─ MultiHeadAttention(Q=K=V=입력)
 ├─ Add & Norm (잔차 연결 + Layer Normalization)
 ├─ Feed-Forward Network
 └─ Add & Norm
출력
```

In [ ]:
class TransformerEncoderBlock(tf.keras.layers.Layer):
    """
    Transformer Encoder Block
    Multi-Head Self-Attention + Add&Norm + FFN + Add&Norm 구조
    """
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        
        # Multi-Head Self-Attention
        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model // num_heads,  # 헤드당 차원
            dropout=dropout_rate
        )
        
        # Feed-Forward Network: d_model → d_ff → d_model
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(d_ff, activation='relu'),  # W1, b1 (ReLU)
            tf.keras.layers.Dense(d_model)                   # W2, b2
        ])
        
        # Layer Normalization (두 개)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Dropout
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, training=False, mask=None):
        # --- Sub-Layer 1: Multi-Head Self-Attention ---
        # Q = K = V = x (Self-Attention)
        attn_output = self.mha(x, x, x, attention_mask=mask, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        # 잔차 연결 + Layer Norm
        out1 = self.layernorm1(x + attn_output)
        
        # --- Sub-Layer 2: Feed-Forward Network ---
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        # 잔차 연결 + Layer Norm
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2  # (batch, seq_len, d_model)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'd_ff': self.d_ff
        })
        return config


# 동작 확인
d_model = 32
num_heads = 4
d_ff = 128  # d_model의 4배
batch_size = 2
seq_len = 10

encoder_block = TransformerEncoderBlock(d_model, num_heads, d_ff)
dummy_input = tf.random.normal((batch_size, seq_len, d_model))
block_output = encoder_block(dummy_input, training=False)

print('Encoder Block 입력 shape:', dummy_input.shape)   # (2, 10, 32)
print('Encoder Block 출력 shape:', block_output.shape)  # (2, 10, 32)
print('입력과 출력 차원 동일:', dummy_input.shape == block_output.shape)

## 4. Transformer Encoder 스택 (여러 Encoder Block 쌓기)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
    """
    토큰 임베딩 + Positional Encoding을 합산하는 레이어
    """
    def __init__(self, vocab_size, d_model, max_seq_len=512, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        # 토큰 임베딩
        self.token_embedding = tf.keras.layers.Embedding(
            input_dim=vocab_size,
            output_dim=d_model
        )
        
        # Positional Encoding (trainable=False 고정값)
        pe_matrix = get_positional_encoding(max_seq_len, d_model)
        self.pos_encoding = tf.cast(pe_matrix, dtype=tf.float32)
    
    def call(self, x):
        seq_len = tf.shape(x)[1]
        # 토큰 임베딩
        embed = self.token_embedding(x)  # (batch, seq_len, d_model)
        # sqrt(d_model)로 스케일링 (임베딩 값을 PE와 비교 가능한 크기로)
        embed *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # Positional Encoding 더하기
        embed += self.pos_encoding[:, :seq_len, :]
        return embed


class TransformerEncoder(tf.keras.Model):
    """
    N개의 Encoder Block을 쌓은 Transformer Encoder
    분류 작업용 헤드 포함
    """
    def __init__(self, vocab_size, d_model, num_heads, d_ff,
                 num_layers, num_classes, max_seq_len=512,
                 dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        
        # 임베딩 + Positional Encoding
        self.pos_embedding = PositionalEmbedding(vocab_size, d_model, max_seq_len)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        
        # N개의 Encoder Block 스택
        self.encoder_blocks = [
            TransformerEncoderBlock(d_model, num_heads, d_ff, dropout_rate,
                                    name=f'encoder_block_{i}')
            for i in range(num_layers)
        ]
        
        # 분류 헤드
        self.global_avg_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.fc1 = tf.keras.layers.Dense(64, activation='relu')
        self.fc_dropout = tf.keras.layers.Dropout(dropout_rate)
        self.classifier = tf.keras.layers.Dense(num_classes,
                                                 activation='sigmoid' if num_classes == 1 else 'softmax')
    
    def call(self, x, training=False):
        # 임베딩 + Positional Encoding
        x = self.pos_embedding(x)
        x = self.dropout(x, training=training)
        
        # N개 Encoder Block 통과
        for block in self.encoder_blocks:
            x = block(x, training=training)
        
        # 시퀀스 차원 평균 풀링
        x = self.global_avg_pool(x)       # (batch, d_model)
        x = self.fc1(x)                   # (batch, 64)
        x = self.fc_dropout(x, training=training)
        return self.classifier(x)          # (batch, num_classes)


# 모델 구조 확인
vocab_size = 10000
d_model_demo = 64
num_heads_demo = 4
num_layers_demo = 2

demo_model = TransformerEncoder(
    vocab_size=vocab_size, d_model=d_model_demo, num_heads=num_heads_demo,
    d_ff=d_model_demo*4, num_layers=num_layers_demo, num_classes=1
)

# Build 하여 파라미터 수 확인
dummy_tokens = tf.ones((1, 20), dtype=tf.int32)
_ = demo_model(dummy_tokens)
demo_model.summary()

## 5. IMDB 텍스트 분류 Transformer

In [ ]:
# 하이퍼파라미터
VOCAB_SIZE = 10000
MAX_LEN = 200
D_MODEL = 64
NUM_HEADS = 4
D_FF = 256          # D_MODEL의 4배
NUM_LAYERS = 2
DROPOUT_RATE = 0.1
BATCH_SIZE = 64
EPOCHS = 5

# IMDB 데이터 로드
print('IMDB 데이터 로딩...')
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(
    num_words=VOCAB_SIZE
)

# 패딩 (최대 길이로 통일)
x_train = keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=MAX_LEN, padding='post', truncating='post'
)
x_test = keras.preprocessing.sequence.pad_sequences(
    x_test, maxlen=MAX_LEN, padding='post', truncating='post'
)

print(f'훈련 데이터: {x_train.shape}, 레이블: {y_train.shape}')
print(f'테스트 데이터: {x_test.shape}, 레이블: {y_test.shape}')
print(f'클래스 분포 (훈련): 긍정={y_train.sum()}, 부정={len(y_train)-y_train.sum()}')

# Transformer 모델 생성
model = TransformerEncoder(
    vocab_size=VOCAB_SIZE + 1,  # +1: 패딩 토큰
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    d_ff=D_FF,
    num_layers=NUM_LAYERS,
    num_classes=1,
    max_seq_len=MAX_LEN,
    dropout_rate=DROPOUT_RATE
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Build
_ = model(x_train[:1])
model.summary()

# 학습
print('\n모델 학습 시작...')
history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
    ]
)

# 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'\n테스트 정확도: {test_acc:.4f}')
print(f'테스트 손실:   {test_loss:.4f}')

# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(history.history['accuracy'], label='훈련 정확도', marker='o')
axes[0].plot(history.history['val_accuracy'], label='검증 정확도', marker='s')
axes[0].set_title('정확도', fontsize=12)
axes[0].set_xlabel('에포크')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['loss'], label='훈련 손실', marker='o')
axes[1].plot(history.history['val_loss'], label='검증 손실', marker='s')
axes[1].set_title('손실', fontsize=12)
axes[1].set_xlabel('에포크')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('IMDB Transformer 학습 곡선', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. 정리

### Transformer Encoder 구성 요소 요약

| 구성 요소 | 역할 | 수식/설명 |
|-----------|------|-----------|
| Token Embedding | 정수 인덱스 → 벡터 | 학습 가능한 임베딩 행렬 |
| Positional Encoding | 위치 정보 부여 | sin/cos 고정값 |
| Multi-Head Attention | 시퀀스 내 관계 모델링 | $h$개 헤드 병렬 Attention |
| Add & Norm | 안정적 학습 | 잔차 연결 + Layer Normalization |
| Feed-Forward Network | 비선형 변환 | ReLU 활성화 |
| Global Avg Pooling | 시퀀스 → 벡터 | 분류 헤드 입력용 |

### 주요 하이퍼파라미터

- **d_model**: 임베딩 차원 (보통 64~512)
- **num_heads**: 헤드 수 (d_model의 약수)
- **d_ff**: FFN 내부 차원 (보통 d_model × 4)
- **num_layers**: Encoder Block 스택 수 (BERT: 12, GPT-2 small: 12)

### 다음 단계

**Chapter 07-03: BERT Fine-Tuning** — 사전학습된 대규모 Transformer(BERT)를 다운로드하여 특정 태스크에 파인튜닝한다.